In [1]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3")

In [8]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

# Ensure Qdrant server is running and accessible
try:
    client = QdrantClient(path="/tmp/langchain_qdrant_demo")
    client.get_collection("demo_collection")
except RuntimeError as e:
    if "already accessed by another instance" in str(e):
        client = QdrantClient(path="/tmp/langchain_qdrant_demo", force_reconnect=True)
    else:
        raise e

try:
    client.get_collection("demo_collection")
except ValueError:
    client.create_collection(
        collection_name="demo_collection",
        vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
    )

vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection",
    embedding=embeddings,
)

RuntimeError: Storage folder /tmp/langchain_qdrant_demo is already accessed by another instance of Qdrant client. If you require concurrent access, use Qdrant server instead.

In [5]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(path="/tmp/langchain_qdrant_demo")

try:
    client.get_collection("demo_collection")
except ValueError:
    client.create_collection(
        collection_name="demo_collection",
        vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
    )

vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection",
    embedding=embeddings,
)

ConnectError: [Errno 61] Connection refused

In [1]:
from langchain_ollama import OllamaLLM

#!ollama pull llama3.1
llm = OllamaLLM(model="llama3.2")

In [2]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt.messages[0].prompt.template

/Users/marcofura/anaconda3/envs/actividad_1/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


"You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"

In [3]:
from openai import OpenAI

In [84]:
def call_openai_api(prompt):
    client = OpenAI()
    print(f"contentPRINT: {prompt.content}")
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            # {"role": "system", "content":context },
            {"role": "user", "content": prompt}
        ],
    )
    return completion.choices[0].message

In [14]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [16]:
from langchain_core.runnables import Runnable


class DebugRunnable(Runnable):
    def __init__(self, step_name):
        self.step_name = step_name

    def invoke(self, input):
        print(f"Step '{self.step_name}' - Input: {input}")
        output = input  # Si quieres modificar el dato, hazlo aquí
        print(f"Step '{self.step_name}' - Output: {output}")
        return output

In [20]:
def debug_step(input, step_name):
    print(f"Step '{step_name}' - Input: {input}")
    return input

In [22]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# retriever = vector_store.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def method(param):
    print(f"param: {param}")
    return param


# RAG chain
rag_chain = (
    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
    | prompt
    | llm
    # | (lambda inputs:call_openai_api(inputs.messages[0]))
    | StrOutputParser()
)

/Users/marcofura/anaconda3/envs/actividad_1/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [62]:
prompt.messages[0].prompt.template

"You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"

In [18]:
context = "Rex es un golden retriever de 8 años, le gustan las frutas dulces como el plátano y las manzanas, aparte es un perro muy grande. No le gusta que le toquen las orejas."

In [23]:
question = "Tengo que acariciar a Rex, que parte de su cuerpo debería evitar?"
respuesta = rag_chain.invoke({"context": context, "question": question})

In [24]:
respuesta

'Deberías evitar acariciar las orejas de Rex, ya que no le gusta que le toquen esa parte de su cuerpo.'